# Tworzenie indeksu za pomocą biblioteki Annoy #
W tym notatniku przedstawimy sposób tworzenia indeksu Annoy na przykładzie bazy faktów benchmarku OpenbookQA

In [1]:
import numpy as np
import random
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer

### Wczytanie bazy danych benchmarku OpenbookQA z pliku txt ###
Aby wczytać fakty z benchmarka openbook należy pobrać ze strony https://allenai.org/data/open-book-qa plik z bazą danych w formacie txt

In [2]:
facts_path = "..\openbook.txt"
with open(facts_path) as f:
    facts = [line[1:-2] for line in f.readlines()]
facts[:5]

['A bee is a pollinating animal',
 'A bird is a pollinating animal',
 'An electrical conductor is a vehicle for the flow of electricity',
 'An example of a change in the Earth is an ocean becoming a wooded area',
 'An example of a chemical change is acid breaking down substances']

### Enkodowanie bazy danych ###
Za pomocą funkcji SentenceTransformer enkodujęmy bazę danych. Należy pamiętać aby model, którym w systemie/bibliotece będzie enkodować przetwarzane pytanie musi być taki sam jak model który zostanie użyty do enkodowania bazy danych

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)
embeddings = model.encode(facts)

print(f"Liczba wymiarów enkodowanej bazy danych: {embeddings.shape[1]}")

Liczba wymiarów enkodowanej bazy danych: 384


### Tworzenie indeksu ###

In [4]:
#AnnoyIndex(f, metric) returns a new index that's read-write and stores vector of f dimensions. Metric can be "angular", "euclidean", "manhattan", "hamming", or "dot".
# Stworzenie obiektu klasy Annoy z podaną ilością wymiarów enkodowanej bazy danych wraz z miarą.
index_annoy = AnnoyIndex(embeddings.shape[1], 'angular')
# Dodanie dokumentów do indeksu
for i, embedding in enumerate(embeddings):
    index_annoy.add_item(i, embedding)

### Dostosowanie parametrów i zapisanie indeksu do pliku ###
W tym momencie kluczowe dla nas jest dobranie odpowiedniej liczby drzew które mają zostać stworzone. Zależnie od tego co chcemy osiągnąć możemy zdecydować się na większą ilość drzew lub mniejsza. Większa ilość drzew zwiększa jakość uzyskiwanych dokumentów, natomiast powoduję ona wzrost rozmiaru indeksu a co za tym idzie jego czas wczytania. Użytkownik musi sam dobrać indeks w zależności od swoich preferencji. Jako zespół polecamy, aby liczba drzew wynosiła około 100. Algorytm działa wtedy bardzo szybko, a jakość dokumentów jest zadowalająca (przynajmniej dla nas).

In [5]:
#Liczba drzew
index_annoy.build(100)

#zapisanie indeksu
index_path = f'testAnnoyIndex.ann'
index_annoy.save(index_path)

True

# Przyklad wczytania modelu:

In [6]:
u = AnnoyIndex(384, 'angular')
u.load('testAnnoyIndex.ann')

True

# Przykładowe wyszukanie K najbliższych dokumentów #

In [7]:
import time

question = "bee"
k = 3
tick = time.process_time()
I = u.get_nns_by_vector(model.encode(question), k)
tock = time.process_time()
print(f"Czas wyszukiwania: {tock-tick} sekund")
print(I)
result_facts = [facts[i] for i in I]
result_facts

Czas wyszukiwania: 0.015625 sekund
[0, 471, 472]


['A bee is a pollinating animal',
 'bees convert nectar into honey',
 'bees eat pollen']